# TODO: Zklend

In [ ]:
from typing import Dict

import IPython.display
import pandas

import classes
import constants
import db

## Load and prepare events

In [ ]:
# Establish the connection.
connection = db.establish_connection()

# Load all Zklend events.
zklend_events = pandas.read_sql(
    sql = 
    f"""
    SELECT
        *
    FROM
        starkscan_events
    WHERE
        from_address='{constants.Protocol.ZKLEND.value}'
    AND
        key_name IN ('Deposit', 'Withdrawal', 'CollateralEnabled', 'CollateralDisabled', 'Borrowing', 'Repayment', 'Liquidation', 'AccumulatorsSync')
    ORDER BY
        block_number, id ASC;
    """,
    con = connection,
)

# Close the connection.
connection.close()

In [ ]:
zklend_events.set_index('id', inplace = True)

# Process events

Events examples:
- [Deposit](https://starkscan.co/event/0x036185142bb51e2c1f5bfdb1e6cef81f8ea87fd4d777990014249bf5435fd31b_3)
- [AccumulatorsSync](https://starkscan.co/event/0x029628b89875a98c1c64ae206e7eb65669cb478a24449f3485f5e98aba6204dc_0)
- [CollateralEnabled](https://starkscan.co/event/0x036185142bb51e2c1f5bfdb1e6cef81f8ea87fd4d777990014249bf5435fd31b_6)
- [CollateralDisabled](https://starkscan.co/event/0x0049b445bed84e0118795dbd22d76610ccac2ad626f8f04a1fc7e38113c2afe7_0)
- [Withdrawal](https://starkscan.co/event/0x03472cf7511687a55bc7247f8765c4bbd2c18b70e09b2a10a77c61f567bfd2cb_4)
- [Borrowing](https://starkscan.co/event/0x076b1615750528635cf0b63ca80986b185acbd20fa37f0f2b5368a4f743931f8_3)
- [Repayment](https://starkscan.co/event/0x06fa3dd6e12c9a66aeacd2eefa5a2ff2915dd1bb4207596de29bd0e8cdeeae66_5)
- [Liquidation](https://starkscan.co/event/0x07b8ec709df1066d9334d56b426c45440ca1f1bb841285a5d7b33f9d1008f256_5)

In [ ]:
# Iterate over ordered events to obtain the final state of each user.
state = classes.State()
for _, event in zklend_events.iterrows():
    state.process_event(event = event)

In [ ]:
prices = classes.Prices()

In [ ]:
# TODO
from typing import Dict
import collections
import decimal


# Source: Starkscan, e.g. 
# https://starkscan.co/token/0x049d36570d4e46f48e99674bd3fcc84644ddd6b96f7c741b1562b82f9e004dc7 for ETH.
TOKEN_DECIMAL_FACTORS = {
    "ETH": decimal.Decimal('1e18'),
    "wBTC": decimal.Decimal('1e8'),
    "USDC": decimal.Decimal('1e6'),
    "DAI": decimal.Decimal('1e18'),
    "USDT": decimal.Decimal('1e6'),
}


# Source: https://zklend.gitbook.io/documentation/using-zklend/technical/asset-parameters.
COLLATERAL_FACTORS = {
    "ETH": decimal.Decimal('0.80'),
    "wBTC": decimal.Decimal('0.70'),
    "USDC": decimal.Decimal('0.80'),
    "DAI": decimal.Decimal('0.70'),
    "USDT": decimal.Decimal('0.70'),
}
# TODO: irrelevant?
BORROW_FACTORS = {
    "ETH": decimal.Decimal('1.00'),
    "wBTC": decimal.Decimal('0.91'),
    "USDC": decimal.Decimal('1.00'),
    "DAI": decimal.Decimal('0.91'),
    "USDT": decimal.Decimal('0.91'),
}


# TODO: compute risk-adjusted collateral and borrowings separately, so that we know how much
# of the borrowings can be repaid?
def compute_health_factor(
    user: str,
    user_state: classes.UserState,
    prices: Dict[str, decimal.Decimal],
) -> decimal.Decimal:
#     # TODO: for debugging
#     if user != '0x3ed214fbc01f7fcfbb3e77bd8d41cfc7dfa403addd672e6990649348585cdc4':
#         return
    # TODO: is it okay to compute health using the values denoted in USD?
    risk_adjusted_collateral_usd = decimal.Decimal('0')
    risk_adjusted_borrowings_usd = decimal.Decimal('0')
    for token, token_state in user_state.token_states.items():
        # TODO: perform the conversion using TOKEN_DECIMAL_FACTORS sooner (in `UserTokenState`?)
        collateral = token_state.collateral_enabled * token_state.deposit / TOKEN_DECIMAL_FACTORS[token]
        borrowings = token_state.borrowings / TOKEN_DECIMAL_FACTORS[token]
        risk_adjusted_collateral_usd += COLLATERAL_FACTORS[token] * collateral * prices[token]
        # TODO: are the liabilities discounted? I guess not? -> no more "risk-adjusted"
        risk_adjusted_borrowings_usd += borrowings * prices[token]
#         risk_adjusted_borrowings_usd += BORROW_FACTORS[token] * borrowings * prices[token]
#         # TODO: for debugging
#         print(
#             token,
#             'col = ', COLLATERAL_FACTORS[token] * collateral * prices[token],
#             'bor = ', token_state.borrowings * prices[token],
#         )
    if risk_adjusted_borrowings_usd == decimal.Decimal('0'):
#         # TODO: assumes collateral is positive
        return decimal.Decimal('Inf')

    health_factor = risk_adjusted_collateral_usd / risk_adjusted_borrowings_usd

    if health_factor < decimal.Decimal('0.9'):
        print(f'Suspiciously low health factor = {health_factor} of user = {user}, investigate.')
#     # TODO: too many loans eligible for liquidation?
#     elif health_factor < decimal.Decimal('1'):
#         print(f'Health factor = {health_factor} of user = {user} eligible for liquidation.')
    return health_factor


# TODO: do we need this?
health_factors = collections.defaultdict(decimal.Decimal)
# TODO: rename?
def compute_health_factors(state: classes.State, prices: Dict[str, decimal.Decimal]):
    for user, user_state in state.user_states.items():
        health_factor = compute_health_factor(user = user, user_state = user_state, prices = prices)

#         # TODO: what does really happen here?
#         # TODO: Explore one of the liquidation events?
#         if health_factor < decimal.Decimal('1'):
#             # TODO: this is the token the price of which drops the most?
#             liquidated_collateral_token = ???
#             # TODO: the amount is calculated in such a way that health <= (i.e. =) 1?
#             liquidated_collateral_amount = ???
#             # TODO: this is one of the tokens that are not liquidated, right? how do we choose one?
#             liquidated_borrowings_token = ???
#             # TODO: this amount represents the same fraction as the fraction of the collateral?
#             liquidated_borrowings_amount = ???

#         # TODO: do we need to save it?
#         health_factors[user] = health_factor

In [ ]:
# TODO
# Compute health for all users in the final state.
compute_health_factors(state = state, prices = prices.prices)

In [ ]:
# TODO
# COL:
# 1.1111 A: p = 100, cf = 0.9
# 1.1111 B: p = 100, cf = 0.9
# 0 C: p = 100, cf = 0.9
# BOR:
# 0 A: p = 100 
# 1 B: p = 100
# 1 C: p = 100
# H = 1

# ->

# COL:
# 1.1111 A: p = 95, cf = 0.9
# 1.1111 B: p = 100, cf = 0.9
# 0 C: p = 100, cf = 0.9
# BOR:
# 0 A: p = 95
# 1 B: p = 100
# 1 C: p = 100
# H = 0.975

# ->

# COL:
# 1 A: p = 95, cf = 0.9
# 1.1111 B: p = 100, cf = 0.9
# 0 C: p = 100, cf = 0.9
# BOR:
# 0 A: p = 95
# 0.9 B: p = 100
# 1 C: p = 100
# H = 0.9763

# ->

# COL:
# 0.5 A: p = 95, cf = 0.9
# 1.1111 B: p = 100, cf = 0.9
# 0 C: p = 100, cf = 0.9
# BOR:
# 0 A: p = 95
# 0.45 B: p = 100
# 1 C: p = 100
# H = 0.9845

# ->

# COL:
# 0 A: p = 95, cf = 0.9
# 1.1111 B: p = 100, cf = 0.9
# 0 C: p = 100, cf = 0.9
# BOR:
# 0 A: p = 95
# 0 B: p = 100
# 1 C: p = 100
# H = 1

# H_coin < 1
# H_rest = 1
# -> 
# coin is liquidated completely .. TODO: for B or C???

In [ ]:
# TODO
# COL:
# 1.1111 A: p = 100, cf = 0.9
# 1.2222 B: p = 100, cf = 0.9
# 0 C: p = 100, cf = 0.9
# BOR:
# 0 A: p = 100 
# 1 B: p = 100
# 1 C: p = 100
# H = 1.05

# ->

# COL:
# 1.1111 A: p = 85, cf = 0.9
# 1.2222 B: p = 100, cf = 0.9
# 0 C: p = 100, cf = 0.9
# BOR:
# 0 A: p = 85
# 1 B: p = 100
# 1 C: p = 100
# H = 0.975

# ->

# COL:
# 1 A: p = 85, cf = 0.9
# 1.2222 B: p = 100, cf = 0.9
# 0 C: p = 100, cf = 0.9
# BOR:
# 0 A: p = 85
# 0.9 B: p = 100
# 1 C: p = 100
# H = 0.9816

# ->

# COL:
# 0.5 A: p = 85, cf = 0.9
# 1.2222 B: p = 100, cf = 0.9
# 0 C: p = 100, cf = 0.9
# BOR:
# 0 A: p = 85
# 0.45 B: p = 100
# 1 C: p = 100
# H = 1.0224

# ->

# COL:
# 0 A: p = 85, cf = 0.9
# 1.2222 B: p = 100, cf = 0.9
# 0 C: p = 100, cf = 0.9
# BOR:
# 0 A: p = 85
# 0 B: p = 100
# 1 C: p = 100
# H = 1.1

# H_coin < 1
# H_rest > 1
# -> 
# coin is liquidated partially

# TODO: Debugging negative deposits below

In [ ]:
# TODO: negative deposits?
print("Action   Token   Raw value   Raw $   Readable $")
for attr in ['deposit', 'borrowings']:
    for token in ['ETH', 'wBTC', 'USDC', 'DAI', 'USDT']:
        values = [getattr(user_state.token_states[token], attr) for user, user_state in state.user_states.items()]
        print(attr, token, pandas.Series(values).min(), f"${prices.to_dollars(pandas.Series(values).min(), token)}", prices.to_dollars_pretty(pandas.Series(values).min(), token))

In [ ]:
# TODO: show negative deposits
[
    (user, user_state.token_states['USDT'].deposit)
    for user, user_state
    in state.user_states.items()
    if user_state.token_states['USDT'].deposit < 0
]